In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
loader = DirectoryLoader("/home/jinzy/work/automation/ask_gpt/testdocs", glob="**/*.pdf", show_progress=True, use_multithreading=True)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
)
txt_docs = loader.load_and_split(text_splitter=text_splitter)

embeddings = OpenAIEmbeddings()
txt_docsearch = Chroma.from_documents(txt_docs, embeddings)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=2048)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.60s/it]


In [4]:
from langchain.chains.summarize import load_summarize_chain

# 创建总结链
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)

In [6]:
chain.run(txt_docs[:5])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"艺赛旗机器人流程挖掘软件（iS-RPM）

Robotic Process Mining

© Copyright 2022 i-search Co.Ltd.

1. 迎接数字时代，激活数据要素潜能

1.1 企业高质量发展面临的问题与挑战

近年来在疫情影响下，企业面临着人力资源紧缺、工作效率等等压力与风险。通过信通 院的调查发现，数字化程度高的企业在风险应对能力方面要高于数字化程度低的企业，这说 明，数字化转型可以提高企业风险应对能力。

随着数字经济的快速发展，以及国家多项政策的不断推进，国内传统企业的数字化转型，

已经从部分头部企业的“可选项”转变为更多行业、更多企业的“必选项”。

在众多数字化转型方案中，流程优化，业务自动化已成为企业高速增长的关键所在。尤 其是 RPA 给企业高质量发展所带来的价值是大家有目共睹的。然而在近几年 RPA 的发展过 程中，我们可以观察到目前整个行业都会面临“重咨询、重交付”的困境。经历 2018 年、 2019 年 RPA 任务自动化的爆发后，2020 年成为任务自动化需求规模化应用的拐点。

Forrester 的调查报告印证了这一点：在 RPA 整个服务过程中，有 60%的 RPA 服务支出 分配给了实施交付领域。RPA 交付人员需要花费大量时间，去和各个自动化需求提出人进行 调研、理解业务，从而造成自动化机会难以发现且交付实施难的局面。

造成这种困局的原因其实和我们当前企业运营模式有一定关系。众多大型企业对于业务 流程的执行角色有着细致且明确的分工，这导致大量的业务环节被分分散在企业内部，由多 名业务人员依次处理，也就是流程碎片化。德勤在最新的全球智能自动化报告中提到：流程

碎片化是目前 RPA 规模化的最大阻碍。碎片化流程中的自动化机会只能由负责操作的人去 发现与识别，这导致需求发现的渠道单一。

为了突破“发现难、交付实施难、规模化突破难”的困局，越来越多的企业开始寻求更 具洞察力的流程发现工具来梳理端到端的

"Original summary:\n\nThe article highlights the importance of digital transformation and process optimization through Robotic Process Automation (RPA) in driving business growth. However, fragmented business processes can make implementing RPA challenging. To overcome this, businesses are turning to process mining tools like iS-RPM, which combines process and task mining to discover repetitive business processes and seamlessly integrate with RPA. iS-RPM utilizes data mining, machine learning, and process modeling to visualize business processes, improve efficiency, and help businesses identify bottlenecks and high-value automation opportunities. By continuously monitoring processes, iS-RPM helps businesses quantify process gaps and improve decision-making. With its mature cross-platform data collection capabilities, iS-RPM provides real-time data collection across various business systems, enabling businesses to achieve RPA scalability, reduce project maintenance costs, and accelerate

In [17]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory


system_template = """
Use the following context to answer the user's question.
If you don't know the answer, say you don't, don't try to make it up. 请用中文回答.
-----------
{context}
-----------
{chat_history}
"""

# 构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('{question}')
]

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_messages(messages)

# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt)
doc_chain = load_qa_chain(llm, chain_type="refine")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain(
    retriever=txt_docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory
)

In [18]:
query = "请用中文解释一下RPM产品中的任务挖掘功能"
result = chain({"question": query})
print(result)

{'question': '请用中文解释一下RPM产品中的任务挖掘功能', 'chat_history': [HumanMessage(content='请用中文解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False), AIMessage(content='iS-RPM产品中的任务挖掘功能是通过收集桌面用户交互数据和从IT系统后台数据中挖掘流程详细信息相关联，打通业务断点，提高流程数据的完整可见性。这意味着企业领导者能够轻松实现海量流程数据的可视化分析，可以看到重复、低效的任务在哪些地方减慢了流程的运行，并在正确的位置分配资源以解决这些问题，为端到端流程优化做出数据驱动的决策。通过任务挖掘，用户可以统计员工花费在各个任务上的时间、执行任务的固定成本，初步分析评估哪些任务更适合自动化。此外，iS-RPM还提供了ROI分析、耗时分析等多个分析面板，帮助分析师深入研究任务，更进一步的了解流程事件，快速验证自动化机会的可行性。iS-RPM是一款机器人流程挖掘软件，将流程挖掘与任务挖掘相结合，通过流程图可视化界面展示企业真实的业务流程，帮助企业360度的全面了解整个企业的工作完成情况，并识别出高重复性的操作，发现企业自动化的机会，为加速企业自动化做能力支撑。', additional_kwargs={}, example=False)], 'answer': 'iS-RPM产品中的任务挖掘功能是通过收集桌面用户交互数据和从IT系统后台数据中挖掘流程详细信息相关联，打通业务断点，提高流程数据的完整可见性。这意味着企业领导者能够轻松实现海量流程数据的可视化分析，可以看到重复、低效的任务在哪些地方减慢了流程的运行，并在正确的位置分配资源以解决这些问题，为端到端流程优化做出数据驱动的决策。通过任务挖掘，用户可以统计员工花费在各个任务上的时间、执行任务的固定成本，初步分析评估哪些任务更适合自动化。此外，iS-RPM还提供了ROI分析、耗时分析等多个分析面板，帮助分析师深入研究任务，更进一步的了解流程事件，快速验证自动化机会的可行性。iS-RPM是一款机器人流程挖掘软件，将流程挖掘与任务挖掘相结合，通过流程图可视化界面展示企业真实的业务流程，帮助企业360度的全面了解整个企业的工作完成情况，并识别出高重复性的操作，发现企业自动化的

In [22]:
query = "请给一个RPM产品的实际案例介绍"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
print(result)
#print(result['answer'])

ValueError: Missing some input keys: {'context'}

In [ ]:
%debug